In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
import warnings
warnings.filterwarnings("ignore")

In [ ]:
test = pd.read_csv('test_encoded.csv')
train = pd.read_csv('train_encoded.csv')

In [ ]:
target = train['target']
#train_id = train['ID']
#test_id = test['ID']

train.drop(['target'], axis=1, inplace=True)
#train.drop(['ID'], axis=1, inplace=True)
#test.drop(['ID'], axis=1, inplace=True)

In [ ]:
print('Train data shape', train.shape)
print('Test data shape', test.shape)

Train data shape (4459, 200)
Test data shape (4000, 200)


In [ ]:
train_scaled = minmax_scale(train, axis = 0)
test_scaled = minmax_scale(test, axis = 0)

#Auto Encoder

In [ ]:
# define the number of features
ncol = train_scaled.shape[1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [ ]:
### Define the encoder dimension
encoding_dim = 200

In [ ]:
input_dim = Input(shape = (ncol, ))

# Encoder Layers
encoded1 = Dense(3000, activation = 'relu')(input_dim)
encoded2 = Dense(2750, activation = 'relu')(encoded1)
encoded3 = Dense(2500, activation = 'relu')(encoded2)
encoded4 = Dense(2250, activation = 'relu')(encoded3)
encoded5 = Dense(2000, activation = 'relu')(encoded4)
encoded6 = Dense(1750, activation = 'relu')(encoded5)
encoded7 = Dense(1500, activation = 'relu')(encoded6)
encoded8 = Dense(1250, activation = 'relu')(encoded7)
encoded9 = Dense(1000, activation = 'relu')(encoded8)
encoded10 = Dense(750, activation = 'relu')(encoded9)
encoded11 = Dense(500, activation = 'relu')(encoded10)
encoded12 = Dense(250, activation = 'relu')(encoded11)
encoded13 = Dense(encoding_dim, activation = 'relu')(encoded12)



In [ ]:
# Decoder Layers
decoded1 = Dense(250, activation = 'relu')(encoded13)
decoded2 = Dense(500, activation = 'relu')(decoded1)
decoded3 = Dense(750, activation = 'relu')(decoded2)
decoded4 = Dense(1000, activation = 'relu')(decoded3)
decoded5 = Dense(1250, activation = 'relu')(decoded4)
decoded6 = Dense(1500, activation = 'relu')(decoded5)
decoded7 = Dense(1750, activation = 'relu')(decoded6)
decoded8 = Dense(2000, activation = 'relu')(decoded7)
decoded9 = Dense(2250, activation = 'relu')(decoded8)
decoded10 = Dense(2500, activation = 'relu')(decoded9)
decoded11 = Dense(2750, activation = 'relu')(decoded10)
decoded12 = Dense(3000, activation = 'relu')(decoded11)
decoded13 = Dense(ncol, activation = 'sigmoid')(decoded12)



In [ ]:
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded13)



In [ ]:
# Compile the Model
autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')

In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 dense (Dense)               (None, 3000)              603000    
                                                                 
 dense_1 (Dense)             (None, 2750)              8252750   
                                                                 
 dense_2 (Dense)             (None, 2500)              6877500   
                                                                 
 dense_3 (Dense)             (None, 2250)              5627250   
                                                                 
 dense_4 (Dense)             (None, 2000)              4502000   
                                                                 
 dense_5 (Dense)             (None, 1750)              350175

In [ ]:
autoencoder.fit(X_train, X_train,batch_size = 32, shuffle = False, validation_data = (X_test, X_test))

126/126 [==============================] - 85s 662ms/step - loss: 0.6931 - val_loss: 0.6930


In [ ]:
encoder = Model(inputs = input_dim, outputs = encoded13)
encoded_input = Input(shape = (encoding_dim, ))

In [ ]:
encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(encoder.predict(test_scaled))
encoded_test = encoded_test.add_prefix('feature_')

In [ ]:
encoded_train['target'] = target

In [ ]:
print(encoded_train.shape)
encoded_train.head()

(4459, 201)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,target
0,0.003509,0.0,0.002897,0.001279,0.0,0.0,0.0,0.002891,0.001612,0.0,0.0,0.005403,0.0,0.0,0.0,0.003828,0.0,0.002883,0.0,0.000151,0.0,0.0,0.0,0.002076,0.001301,0.001685,0.003717,0.001481,0.0,0.001064,0.003822,0.000000,0.008990,0.0,0.000000,0.0,0.006697,0.0,0.0,0.000156,...,0.002441,0.0,0.0,0.0,0.0,0.000000,0.003624,0.0,0.0,0.0,0.0,0.0,0.0,0.003064,0.0,0.003033,0.004319,0.003447,0.005898,0.0,0.0,0.003166,0.000587,0.0,0.0,0.002764,0.0,0.0,0.0,0.010477,0.0,0.0,0.0,0.0,0.004792,0.003436,0.000981,0.0,0.002775,38000000.0
1,0.003503,0.0,0.002900,0.001472,0.0,0.0,0.0,0.002690,0.001814,0.0,0.0,0.005205,0.0,0.0,0.0,0.003924,0.0,0.002720,0.0,0.000000,0.0,0.0,0.0,0.001996,0.001564,0.001686,0.003801,0.001343,0.0,0.001019,0.003752,0.000000,0.009135,0.0,0.000000,0.0,0.006727,0.0,0.0,0.000000,...,0.002218,0.0,0.0,0.0,0.0,0.000000,0.003956,0.0,0.0,0.0,0.0,0.0,0.0,0.002912,0.0,0.003146,0.004292,0.003783,0.005371,0.0,0.0,0.003181,0.000723,0.0,0.0,0.002798,0.0,0.0,0.0,0.010201,0.0,0.0,0.0,0.0,0.004923,0.003326,0.000831,0.0,0.002803,600000.0
2,0.004314,0.0,0.002786,0.002292,0.0,0.0,0.0,0.003277,0.000000,0.0,0.0,0.008530,0.0,0.0,0.0,0.004400,0.0,0.004685,0.0,0.001065,0.0,0.0,0.0,0.002567,0.002187,0.002076,0.002788,0.000945,0.0,0.000000,0.004494,0.001081,0.011811,0.0,0.001087,0.0,0.007561,0.0,0.0,0.000000,...,0.001417,0.0,0.0,0.0,0.0,0.000000,0.004183,0.0,0.0,0.0,0.0,0.0,0.0,0.004195,0.0,0.004788,0.005704,0.002995,0.008334,0.0,0.0,0.004905,0.000000,0.0,0.0,0.004343,0.0,0.0,0.0,0.012463,0.0,0.0,0.0,0.0,0.006520,0.003029,0.000562,0.0,0.005061,10000000.0
3,0.004298,0.0,0.002789,0.002383,0.0,0.0,0.0,0.003250,0.000000,0.0,0.0,0.008535,0.0,0.0,0.0,0.004422,0.0,0.004793,0.0,0.000998,0.0,0.0,0.0,0.002527,0.002215,0.002126,0.002757,0.001022,0.0,0.000000,0.004518,0.001060,0.011845,0.0,0.001113,0.0,0.007690,0.0,0.0,0.000000,...,0.001343,0.0,0.0,0.0,0.0,0.000000,0.004223,0.0,0.0,0.0,0.0,0.0,0.0,0.004196,0.0,0.004818,0.005765,0.003094,0.008313,0.0,0.0,0.004929,0.000000,0.0,0.0,0.004391,0.0,0.0,0.0,0.012445,0.0,0.0,0.0,0.0,0.006540,0.003176,0.000595,0.0,0.005079,2000000.0
4,0.004771,0.0,0.002560,0.001529,0.0,0.0,0.0,0.002261,0.000377,0.0,0.0,0.006699,0.0,0.0,0.0,0.004122,0.0,0.003947,0.0,0.000693,0.0,0.0,0.0,0.002120,0.001732,0.001932,0.003262,0.000734,0.0,0.000456,0.004001,0.000264,0.010367,0.0,0.001069,0.0,0.006745,0.0,0.0,0.000000,...,0.001945,0.0,0.0,0.0,0.0,0.000014,0.003299,0.0,0.0,0.0,0.0,0.0,0.0,0.003285,0.0,0.003590,0.004660,0.002946,0.007818,0.0,0.0,0.003474,0.000000,0.0,0.0,0.003124,0.0,0.0,0.0,0.011309,0.0,0.0,0.0,0.0,0.004851,0.003258,0.000857,0.0,0.003627,14400000.0


In [ ]:
print(encoded_test.shape)
encoded_test.head()

(4000, 200)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199
0,0.001472,0.0,0.002182,0.002471,0.0,0.0,0.0,0.002149,0.002489,0.0,0.0,0.003979,0.0,0.0,0.0,0.003071,0.0,0.002434,0.000000,0.000000,0.0,0.0,0.0,0.002046,0.000890,0.001632,0.002349,0.001488,0.0,0.000193,0.004195,0.000000,0.008082,0.0,0.000000,0.0,0.005713,0.0,0.0,0.000000,...,0.004066,0.001020,0.0,0.0,0.0,0.0,0.000000,0.003037,0.00000,0.0,0.0,0.0,0.0,0.0,0.001730,0.0,0.001631,0.002619,0.004054,0.004392,0.0,0.0,0.002793,0.000331,0.0,0.0,0.003049,0.0,0.0,0.0,0.007485,0.0,0.0,0.0,0.0,0.004093,0.002199,0.000000,0.0,0.002529
1,0.004820,0.0,0.002616,0.002310,0.0,0.0,0.0,0.003553,0.000000,0.0,0.0,0.008365,0.0,0.0,0.0,0.004420,0.0,0.005320,0.000000,0.001140,0.0,0.0,0.0,0.002509,0.001932,0.001369,0.002571,0.000562,0.0,0.000000,0.004859,0.001077,0.013156,0.0,0.000808,0.0,0.008006,0.0,0.0,0.000000,...,0.003058,0.001636,0.0,0.0,0.0,0.0,0.000000,0.004104,0.00000,0.0,0.0,0.0,0.0,0.0,0.004677,0.0,0.004889,0.005177,0.003487,0.008228,0.0,0.0,0.005385,0.000000,0.0,0.0,0.005610,0.0,0.0,0.0,0.013338,0.0,0.0,0.0,0.0,0.006942,0.003259,0.000963,0.0,0.005188
2,0.005173,0.0,0.002409,0.001836,0.0,0.0,0.0,0.002323,0.000000,0.0,0.0,0.007162,0.0,0.0,0.0,0.004181,0.0,0.004250,0.000000,0.001020,0.0,0.0,0.0,0.002175,0.001916,0.001514,0.002782,0.000393,0.0,0.000256,0.004262,0.000203,0.011554,0.0,0.000944,0.0,0.007208,0.0,0.0,0.000000,...,0.003096,0.001891,0.0,0.0,0.0,0.0,0.000277,0.003529,0.00021,0.0,0.0,0.0,0.0,0.0,0.004022,0.0,0.004609,0.004547,0.003309,0.007849,0.0,0.0,0.004513,0.000000,0.0,0.0,0.004512,0.0,0.0,0.0,0.012017,0.0,0.0,0.0,0.0,0.005527,0.003344,0.000974,0.0,0.003941
3,0.003497,0.0,0.002706,0.001941,0.0,0.0,0.0,0.001867,0.001266,0.0,0.0,0.005622,0.0,0.0,0.0,0.003388,0.0,0.003170,0.000229,0.000716,0.0,0.0,0.0,0.001676,0.001451,0.001915,0.003043,0.001259,0.0,0.000494,0.003471,0.000000,0.009930,0.0,0.000121,0.0,0.006328,0.0,0.0,0.000000,...,0.003463,0.002095,0.0,0.0,0.0,0.0,0.000000,0.003493,0.00000,0.0,0.0,0.0,0.0,0.0,0.002833,0.0,0.002616,0.003670,0.002984,0.005283,0.0,0.0,0.003365,0.000729,0.0,0.0,0.003445,0.0,0.0,0.0,0.010294,0.0,0.0,0.0,0.0,0.004008,0.002691,0.000558,0.0,0.003024
4,0.004230,0.0,0.002594,0.001851,0.0,0.0,0.0,0.002148,0.001087,0.0,0.0,0.005855,0.0,0.0,0.0,0.003409,0.0,0.003361,0.000000,0.000584,0.0,0.0,0.0,0.001779,0.001598,0.001966,0.003243,0.000995,0.0,0.000533,0.003773,0.000000,0.010177,0.0,0.000558,0.0,0.006540,0.0,0.0,0.000151,...,0.003502,0.002262,0.0,0.0,0.0,0.0,0.000000,0.003341,0.00000,0.0,0.0,0.0,0.0,0.0,0.003243,0.0,0.003009,0.003508,0.002956,0.005693,0.0,0.0,0.003533,0.000605,0.0,0.0,0.003210,0.0,0.0,0.0,0.010581,0.0,0.0,0.0,0.0,0.004232,0.003250,0.001062,0.0,0.003360


In [ ]:
encoded_train.to_csv('train_encoded.csv', index=False)
encoded_test.to_csv('test_encoded.csv', index=False)